In [64]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

root_dir = "/your/path"

In [98]:
table2 = pd.read_excel(f"{root_dir}/data/Table2_dist.xlsx")
table3 = pd.read_excel(f"{root_dir}/data/Table3_dist.xlsx")

In [99]:
table2

,T2_T,T2_R,FLAIR_T,FLAIR_R,DWI_T,DWI_R,ADC_T,ADC_R,T1_T,T1_R,T1CE_T,T1CE_R
0,1286,1.529,1311,1.341,1175,1.088,1.009,1.771,473,0.84,892,1.595
1,-,-,648,-,-,-,-,-,-,-,1309.5,-
2,1423.2,-,-,-,-,-,-,-,-,-,-,-
3,2290.2,-,-,-,-,-,2,-,-,-,1492.5,-
4,-,-,-,-,544.23,-,2,-,-,-,-,0.781


In [100]:
table3

,T2_T,T2_R,FLAIR_T,FLAIR_R,DWI_T,DWI_R,ADC_T,ADC_R,T1_T,T1_R,T1CE_T,T1CE_R
0,1534,1.638,1107,1.141,1883,1.614,0.54,0.87,513,0.842,818,1.327
1,-,-,-,0.742,-,-,-,-,-,-,-,-
2,-,-,2493,-,-,-,-,2.316,-,-,-,-
3,-,2.61,-,-,-,-,-,2.892,-,-,-,-
4,-,-,-,-,-,-,2.05,2.917,-,-,-,-
5,1778,2.297,1085,1.143,809,0.805,1.879,2.685,439,0.57,747,1.125
6,-,0.967,-,0.608,-,-,0.4,-,-,0.535,-,-
7,913.6,0.968,-,-,402.32,0.476,0.4,-,-,-,-,-
8,-,-,-,-,-,1.025,-,-,-,-,-,-
9,-,1.072,-,-,-,-,-,-,-,-,-,0.768


In [101]:
# Load the dataset into a Pandas DataFrame
df = table2.copy()

# Flatten the DataFrame into a 1D array, excluding the '-' values
values = df.values.flatten()
values = [float(value) if value != '-' else np.nan for value in values]
values = np.array(values).reshape(-1, 1)

# Apply standard scaling to the non-'-' values
scaler = StandardScaler()
scaled_values = scaler.fit_transform(values)

# Reshape the DataFrame and mask to match the scaled values array
reshaped_df = df.values.reshape(-1, 1)
mask = reshaped_df != '-'

# Assign the scaled values back to the DataFrame using the reshaped mask
reshaped_df[mask] = scaled_values[mask]

# Reshape the modified DataFrame back to its original shape
df.iloc[:, :] = reshaped_df.reshape(df.shape)

# Print the modified dataset
print(df)



       T2_T      T2_R   FLAIR_T   FLAIR_R     DWI_T     DWI_R     ADC_T  \
0  0.984769 -0.892821  1.021313 -0.893096  0.822514 -0.893466 -0.893582   
1         -         -  0.052165         -         -         -         -   
2  1.185323         -         -         -         -         -         -   
3   2.45267         -         -         -         -         - -0.892133   
4         -         -         -         - -0.099522         - -0.892133   

      ADC_R      T1_T      T1_R    T1CE_T    T1CE_R  
0 -0.892468 -0.203643 -0.893829  0.408835 -0.892725  
1         -         -         -   1.01912         -  
2         -         -         -         -         -  
3         -         -         -  1.286623         -  
4         -         -         -         - -0.893915  


In [102]:
# Extract the baseline row
baseline_row = df.iloc[0]

# Create a list to store the distances
distances = []

# Iterate over each row (excluding the baseline row)
for index, row in df.iloc[1:].iterrows():
    # Initialize the distance as 0
    dist = 0
    
    # Iterate over each column
    for column in df.columns:
        # Get the values from the current row and the baseline row
        curr_value = row[column]
        baseline_value = baseline_row[column]
        
        # If either value is '-', consider it as 0
        if curr_value == '-' or baseline_value == '-':
            curr_value = 0
            baseline_value = 0
        
        # Calculate the squared difference and add it to the distance
        diff = float(curr_value) - float(baseline_value)
        dist += diff ** 2
    
    # Calculate the square root of the distance and store it in the list
    dist = np.sqrt(dist)
    distances.append(dist)

# Add the distances as a new column in the dataset
df['Distance'] = [0] + distances

# Print the modified dataset
print(df)

       T2_T      T2_R   FLAIR_T   FLAIR_R     DWI_T     DWI_R     ADC_T  \
0  0.984769 -0.892821  1.021313 -0.893096  0.822514 -0.893466 -0.893582   
1         -         -  0.052165         -         -         -         -   
2  1.185323         -         -         -         -         -         -   
3   2.45267         -         -         -         -         - -0.892133   
4         -         -         -         - -0.099522         - -0.892133   

      ADC_R      T1_T      T1_R    T1CE_T    T1CE_R  Distance  
0 -0.892468 -0.203643 -0.893829  0.408835 -0.892725  0.000000  
1         -         -         -   1.01912         -  1.145293  
2         -         -         -         -         -  0.200554  
3         -         -         -  1.286623         -  1.710335  
4         -         -         -         - -0.893915  0.922037  


In [103]:
df.to_excel(f"{root_dir}/data/Table2_dist_scaled.xlsx", index=False)

In [104]:
df = table3[:5]
df

,T2_T,T2_R,FLAIR_T,FLAIR_R,DWI_T,DWI_R,ADC_T,ADC_R,T1_T,T1_R,T1CE_T,T1CE_R
0,1534,1.638,1107,1.141,1883,1.614,0.54,0.87,513,0.842,818,1.327
1,-,-,-,0.742,-,-,-,-,-,-,-,-
2,-,-,2493,-,-,-,-,2.316,-,-,-,-
3,-,2.61,-,-,-,-,-,2.892,-,-,-,-
4,-,-,-,-,-,-,2.05,2.917,-,-,-,-


In [105]:
# Load the dataset into a Pandas DataFrame
df = table3[:5].copy()

# Flatten the DataFrame into a 1D array, excluding the '-' values
values = df.values.flatten()
values = [float(value) if value != '-' else np.nan for value in values]
values = np.array(values).reshape(-1, 1)

# Apply standard scaling to the non-'-' values
scaler = StandardScaler()
scaled_values = scaler.fit_transform(values)

# Reshape the DataFrame and mask to match the scaled values array
reshaped_df = df.values.reshape(-1, 1)
mask = reshaped_df != '-'

# Assign the scaled values back to the DataFrame using the reshaped mask
reshaped_df[mask] = scaled_values[mask]

# Reshape the modified DataFrame back to its original shape
df.iloc[:, :] = reshaped_df.reshape(df.shape)

# Print the modified dataset
print(df)


       T2_T      T2_R   FLAIR_T   FLAIR_R     DWI_T    DWI_R     ADC_T  \
0  1.465048 -0.587978  0.892963 -0.588644  1.932631 -0.58801 -0.589449   
1         -         -         - -0.589178         -        -         -   
2         -         -  2.749896         -         -        -         -   
3         - -0.586676         -         -         -        -         -   
4         -         -         -         -         -        - -0.587426   

      ADC_R      T1_T      T1_R    T1CE_T    T1CE_R  
0 -0.589007  0.097134 -0.589044  0.505767 -0.588395  
1         -         -         -         -         -  
2  -0.58707         -         -         -         -  
3 -0.586298         -         -         -         -  
4 -0.586264         -         -         -         -  


In [106]:
# Extract the baseline row
baseline_row = df.iloc[0]

# Create a list to store the distances
distances = []

# Iterate over each row (excluding the baseline row)
for index, row in df.iloc[1:].iterrows():
    # Initialize the distance as 0
    dist = 0
    
    # Iterate over each column
    for column in df.columns:
        # Get the values from the current row and the baseline row
        curr_value = row[column]
        baseline_value = baseline_row[column]
        
        # If either value is '-', consider it as 0
        if curr_value == '-' or baseline_value == '-':
            curr_value = 0
            baseline_value = 0
        
        # Calculate the squared difference and add it to the distance
        diff = float(curr_value) - float(baseline_value)
        dist += diff ** 2
    
    # Calculate the square root of the distance and store it in the list
    dist = np.sqrt(dist)
    distances.append(dist)

# Add the distances as a new column in the dataset
df['Distance'] = [0] + distances

# Print the modified dataset
print(df)

       T2_T      T2_R   FLAIR_T   FLAIR_R     DWI_T    DWI_R     ADC_T  \
0  1.465048 -0.587978  0.892963 -0.588644  1.932631 -0.58801 -0.589449   
1         -         -         - -0.589178         -        -         -   
2         -         -  2.749896         -         -        -         -   
3         - -0.586676         -         -         -        -         -   
4         -         -         -         -         -        - -0.587426   

      ADC_R      T1_T      T1_R    T1CE_T    T1CE_R  Distance  
0 -0.589007  0.097134 -0.589044  0.505767 -0.588395  0.000000  
1         -         -         -         -         -  0.000535  
2  -0.58707         -         -         -         -  1.856934  
3 -0.586298         -         -         -         -  0.003006  
4 -0.586264         -         -         -         -  0.003408  


In [107]:
df.to_excel(f"{root_dir}/data/Table3MB_dist_scaled.xlsx", index=False)

In [108]:
# Load the dataset into a Pandas DataFrame
df = table3[5:10].copy()

# Flatten the DataFrame into a 1D array, excluding the '-' values
values = df.values.flatten()
values = [float(value) if value != '-' else np.nan for value in values]
values = np.array(values).reshape(-1, 1)

# Apply standard scaling to the non-'-' values
scaler = StandardScaler()
scaled_values = scaler.fit_transform(values)

# Reshape the DataFrame and mask to match the scaled values array
reshaped_df = df.values.reshape(-1, 1)
mask = reshaped_df != '-'

# Assign the scaled values back to the DataFrame using the reshaped mask
reshaped_df[mask] = scaled_values[mask]

# Reshape the modified DataFrame back to its original shape
df.iloc[:, :] = reshaped_df.reshape(df.shape)

# Print the modified dataset
print(df)


       T2_T      T2_R   FLAIR_T   FLAIR_R     DWI_T     DWI_R     ADC_T  \
5  3.283587 -0.552346  1.786545 -0.554839  1.190321 -0.555569 -0.553249   
6         - -0.555219         - -0.555995         -         - -0.556444   
7  1.416281 -0.555217         -         -  0.311797  -0.55628 -0.556444   
8         -         -         -         -         - -0.555094         -   
9         - -0.554992         -         -         -         -         -   

      ADC_R      T1_T      T1_R    T1CE_T    T1CE_R  
5 -0.551508  0.391034 -0.556077  1.056386 -0.554878  
6         -         - -0.556152         -         -  
7         -         -         -         -         -  
8         -         -         -         -         -  
9         -         -         -         - -0.555649  


In [109]:
# Extract the baseline row
baseline_row = df.iloc[0]

# Create a list to store the distances
distances = []

# Iterate over each row (excluding the baseline row)
for index, row in df.iloc[1:].iterrows():
    # Initialize the distance as 0
    dist = 0
    
    # Iterate over each column
    for column in df.columns:
        # Get the values from the current row and the baseline row
        curr_value = row[column]
        baseline_value = baseline_row[column]
        
        # If either value is '-', consider it as 0
        if curr_value == '-' or baseline_value == '-':
            curr_value = 0
            baseline_value = 0
        
        # Calculate the squared difference and add it to the distance
        diff = float(curr_value) - float(baseline_value)
        dist += diff ** 2
    
    # Calculate the square root of the distance and store it in the list
    dist = np.sqrt(dist)
    distances.append(dist)

# Add the distances as a new column in the dataset
df['Distance'] = [0] + distances

# Print the modified dataset
print(df)

       T2_T      T2_R   FLAIR_T   FLAIR_R     DWI_T     DWI_R     ADC_T  \
5  3.283587 -0.552346  1.786545 -0.554839  1.190321 -0.555569 -0.553249   
6         - -0.555219         - -0.555995         -         - -0.556444   
7  1.416281 -0.555217         -         -  0.311797  -0.55628 -0.556444   
8         -         -         -         -         - -0.555094         -   
9         - -0.554992         -         -         -         -         -   

      ADC_R      T1_T      T1_R    T1CE_T    T1CE_R  Distance  
5 -0.551508  0.391034 -0.556077  1.056386 -0.554878  0.000000  
6         -         - -0.556152         -         -  0.004450  
7         -         -         -         -         -  2.063651  
8         -         -         -         -         -  0.000475  
9         -         -         -         - -0.555649  0.002756  


In [110]:
df.to_excel(f"{root_dir}/data/Table3PA_dist_scaled.xlsx", index=False)

In [111]:
# Load the dataset into a Pandas DataFrame
df = table3[10:15].copy()

# Flatten the DataFrame into a 1D array, excluding the '-' values
values = df.values.flatten()
values = [float(value) if value != '-' else np.nan for value in values]
values = np.array(values).reshape(-1, 1)

# Apply standard scaling to the non-'-' values
scaler = StandardScaler()
scaled_values = scaler.fit_transform(values)

# Reshape the DataFrame and mask to match the scaled values array
reshaped_df = df.values.reshape(-1, 1)
mask = reshaped_df != '-'

# Assign the scaled values back to the DataFrame using the reshaped mask
reshaped_df[mask] = scaled_values[mask]

# Reshape the modified DataFrame back to its original shape
df.iloc[:, :] = reshaped_df.reshape(df.shape)

# Print the modified dataset
print(df)


        T2_T      T2_R   FLAIR_T   FLAIR_R     DWI_T     DWI_R     ADC_T  \
10  2.048204 -0.735394  1.136792 -0.736266  1.074836 -0.737106 -0.735613   
11  0.893695         -  0.849836 -0.737114         -         -  -0.73739   
12  0.664456 -0.736724  2.553638         -         -         - -0.737651   
13         -         -         -         -         -         - -0.735661   
14         -         -         - -0.736979         -         -         -   

       ADC_R      T1_T      T1_R    T1CE_T    T1CE_R  
10 -0.734703 -0.130054 -0.737157 -0.206684 -0.737142  
11 -0.737101  1.583531 -0.737271   0.04424 -0.736966  
12         -         -         -         -         -  
13         -         -         -  1.273748         -  
14         -         -         -         -         -  


In [112]:
# Extract the baseline row
baseline_row = df.iloc[0]

# Create a list to store the distances
distances = []

# Iterate over each row (excluding the baseline row)
for index, row in df.iloc[1:].iterrows():
    # Initialize the distance as 0
    dist = 0
    
    # Iterate over each column
    for column in df.columns:
        # Get the values from the current row and the baseline row
        curr_value = row[column]
        baseline_value = baseline_row[column]
        
        # If either value is '-', consider it as 0
        if curr_value == '-' or baseline_value == '-':
            curr_value = 0
            baseline_value = 0
        
        # Calculate the squared difference and add it to the distance
        diff = float(curr_value) - float(baseline_value)
        dist += diff ** 2
    
    # Calculate the square root of the distance and store it in the list
    dist = np.sqrt(dist)
    distances.append(dist)

# Add the distances as a new column in the dataset
df['Distance'] = [0] + distances

# Print the modified dataset
print(df)

        T2_T      T2_R   FLAIR_T   FLAIR_R     DWI_T     DWI_R     ADC_T  \
10  2.048204 -0.735394  1.136792 -0.736266  1.074836 -0.737106 -0.735613   
11  0.893695         -  0.849836 -0.737114         -         -  -0.73739   
12  0.664456 -0.736724  2.553638         -         -         - -0.737651   
13         -         -         -         -         -         - -0.735661   
14         -         -         - -0.736979         -         -         -   

       ADC_R      T1_T      T1_R    T1CE_T    T1CE_R  Distance  
10 -0.734703 -0.130054 -0.737157 -0.206684 -0.737142  0.000000  
11 -0.737101  1.583531 -0.737271   0.04424 -0.736966  2.101090  
12         -         -         -         -         -  1.980458  
13         -         -         -  1.273748         -  1.480432  
14         -         -         -         -         -  0.000712  


In [113]:
df.to_excel(f"{root_dir}/data/Table3BG_dist_scaled.xlsx", index=False)